In [3]:
import pickle

In [4]:
with open('financial_statements.pickle', 'rb') as fp:
    data = pickle.load(fp)

In [5]:
[x for x in data[0]]

['SimFin ID',
 'company',
 'ticker',
 'financial year end (month)',
 'industry code',
 'Share Price',
 'Common Shares Outstanding',
 'Avg. Basic Shares Outstanding',
 'Avg. Diluted Shares Outstanding',
 'Revenues',
 'COGS',
 'SG&A',
 'EBIT',
 'EBITDA',
 'Net Income from Discontinued Op.',
 'Net Profit',
 'Dividends',
 'Cash and Cash Equivalents',
 'Receivables',
 'Current Assets',
 'Net PP&E',
 'Intangible Assets',
 'Goodwill',
 'Total Noncurrent Assets',
 'Total Assets',
 'Short term debt',
 'Accounts Payable',
 'Current Liabilities',
 'Long Term Debt',
 'Total Noncurrent Liabilities',
 'Total Liabilities',
 'Preferred Equity',
 'Share Capital',
 'Treasury Stock',
 'Retained Earnings',
 'Equity Before Minorities',
 'Minorities',
 'Total Equity',
 'Cash From Operating Activities',
 'Cash From Investing Activities',
 'Cash From Financing Activities',
 'Net Change in Cash']

In [6]:
for obj in data:
    for key in obj:
        if isinstance(obj[key], list):
            for i in range(len(obj[key])):
                try:
                    obj[key][i] = float(obj[key][i])
                except ValueError:
                    obj[key][i] = None

In [7]:
[x for x in data[0]['EBIT'] if x]

[14.238,
 17.968,
 21.469,
 23.638,
 28.293,
 25.786,
 24.907,
 26.846,
 19.156,
 23.706,
 24.159,
 70.293,
 56.935,
 37.617,
 21.974,
 41.524,
 44.526,
 43.282,
 42.444,
 44.612,
 41.563,
 46.359,
 49.865,
 37.78,
 41.626,
 41.048,
 37.685]

In [2]:
# X41	total liabilities / ((profit on operating activities + depreciation) * (12/365)) 
# X42	profit on operating activities / sales 
# X43	rotation receivables + inventory turnover in days 
# X44	(receivables * 365) / sales 
# X45	net profit / inventory 
# X46	(current assets - inventory) / short-term liabilities 
# X47	(inventory * 365) / cost of products sold 
# X48	EBITDA (profit on operating activities - depreciation) / total assets 
# X49	EBITDA (profit on operating activities - depreciation) / sales 
# X50	current assets / total liabilities 
# X51	short-term liabilities / total assets 
# X52	(short-term liabilities * 365) / cost of products sold) 
# X53	equity / fixed assets 
# X54	constant capital / fixed assets 
# X55	working capital 
# X56	(sales - cost of products sold) / sales 
# X57	(current assets - inventory - short-term liabilities) / (sales - gross profit - depreciation) 
# X58	total costs /total sales 
# X59	long-term liabilities / equity 
# X60	sales / inventory 
# X61	sales / receivables 
# X62	(short-term liabilities *365) / sales 
# X63	sales / short-term liabilities 
# X64	sales / fixed assets

def get_last_24_features(obj):
    def catchError(obj, i, func):
        try:
            return func(obj, i)
        except (TypeError, ZeroDivisionError):
            return '';
    funcs = {
        
         # X1:net profit / total assets 
        'X1': lambda obj, i: obj['Net Profit'][i] / obj['Total Assets'][i],
        
        # X2:total liabilities / total assets 
        'X2': lambda obj, i: obj['Total Liabilities'][i] / obj['Total Assets'][i],
        
        # X3:working capital / total assets 
        # working capital = current assets - current liabilities 
        'X3': lambda obj, i: (obj['Current Assets'][i] - obj['Current Liabilities'][i]) / obj['Total Assets'][i], 
        
        # X4:current assets / short-term liabilities 
        # assume short term liability = current liability
        'X4': lambda obj, i:  obj['Current Assets'][i] / obj['Current Liabilities'][i], 
        
        # X5:[(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365
        # cash + short-term securities + receivables = current assets
        #  Operating expenses - depreciation = sg&A   
        'X5': lambda obj, i: (obj['Current Assets'][i]-obj['Current Liabilities'][i]) / obj['SG&A'][i] * 365,
        
        # X6:retained earnings / total assets 
        'X6': lambda obj, i: obj['Retained Earnings'][i] / obj['Total Assets'][i],
        
        # X7:EBIT / total assets 
        'X7': lambda obj, i: obj['EBIT'][i] / obj['Total Assets'][i],
        
        # X8:book value of equity / total liabilities 
        # book value of equity = total assests - total liabilities
        'X8': lambda obj, i: (obj['Total Assets'][i]-obj['Total Liabilities'][i]) / obj['Total Liabilities'][i], 
        
        # X9:sales / total assets 
        'X9': lambda obj, i: obj['Revenues'][i] / obj['Total Assets'][i],
        
        # X10:equity / total assets 
        'X10': lambda obj, i: obj['Total Equity'][i] / obj['Total Assets'][i], 
        
        # X11:(gross profit + extraordinary items + financial expenses) / total assets
        # extroordinary items and financial expenses missing
        'X11': lambda obj, i: '', 
        
        # X12:gross profit / short-term liabilities
        # gross profit = revenue - cogs 
        'X12': lambda obj, i: (obj['Revenues'][i]-obj['COGS'][i]) / obj['Current Liabilities'][i],
       
        # X13:(gross profit + depreciation) / sales 
        # depreciation = operating expenses - sg&A = gross profit - net income - sg&A
        # gross profit + depreciation = 2 * gross profit -net income - sg&A
        'X13': lambda obj, i: (2 * (obj['Revenues'][i]-obj['COGS'][i]) - obj['Net Profit'][i] - obj['SG&A'][i]) / obj['Revenues'][i],
        
        # X14:(gross profit + interest) / total assets 
        # interest missing
        'X14': lambda obj, i:  '', 
        
        # X15:(total liabilities * 365) / (gross profit + depreciation) 
        # gross profit + depreciation = 2 * gross profit -net income - sg&A
        'X15': lambda obj, i: (obj['Total Liabilities'][i] * 365) / (2 * (obj['Revenues'][i]-obj['COGS'][i]) - obj['Net Profit'][i] - obj['SG&A'][i]), 
        
        # X16:(gross profit + depreciation) / total liabilities 
        'X16': lambda obj, i: (2 * (obj['Revenues'][i]-obj['COGS'][i]) - obj['Net Profit'][i] - obj['SG&A'][i]) / obj['Total Liabilities'][i], 
        
        # X17:total assets / total liabilities 
        'X17': lambda obj, i: obj['Tota Assets'][i] / obj['Toal Liabilities'][i],
        
        # X18:gross profit / total assets
        'X18': lambda obj, i: (obj['Revenues'][i]-obj['COGS'][i]) / obj['Total Assets'][i],
        
        # X19:gross profit / sales 
        'X19': lambda obj, i: (obj['Revenues'][i]-obj['COGS'][i]) / obj['Revenues'][i],
        
         # X20:(inventory * 365) / sales
        # inventory missing
        'X20': lambda obj, i: '', 
        
        'x41': lambda obj, i: obj['Total Liabilities'][i] / (obj['EBIT'][i] * (12.0/365.0)),
        'x42': lambda obj, i: obj['EBIT'][i] / obj['Revenues'][i],
        'x43': lambda obj, i: '', # inventory missing
        'x44': lambda obj, i: obj['Receivables'][i] * 365 / obj['Revenues'][i],
        'x45': lambda obj, i: '', # inventory missing
        'x46': lambda obj, i: '', # inventory missing
        'x47': lambda obj, i: '', # inventory missing
        'x48': lambda obj, i: obj['EBITDA'][i] / obj['Total Assets'][i],
        'x49': lambda obj, i: obj['EBITDA'][i] / obj['Revenues'][i],
        'x50': lambda obj, i: obj['Current Assets'][i] / obj['Total Liabilities'][i],
        'x51': lambda obj, i: obj['Current Liabilities'][i] / obj['Total Assets'][i], # assume short term liability === current liability
        'x52': lambda obj, i: obj['Current Liabilities'][i] * 365 / obj['COGS'][i],
        'x53': lambda obj, i: obj['Total Equity'][i] / obj['Total Noncurrent Assets'][i],
        'x54': lambda obj, i: obj['Net PP&E'][i] / obj['Total Noncurrent Assets'][i],
        'x55': lambda obj, i: obj['Current Assets'][i] - obj['Current Liabilities'][i],
        'x56': lambda obj, i: (obj['Revenues'][i] - obj['COGS'][i]) / obj['Revenues'][i],
        'x57': lambda obj, i: '', # inventory missing
        'x58': lambda obj, i: (obj['Revenues'][i] - obj['Net Profit'][i]) / obj['Revenues'][i],
        'x59': lambda obj, i: obj['Total Noncurrent Liabilities'][i] / obj['Total Equity'][i],
        'x60': lambda obj, i: '', # inventory missing
        'x61': lambda obj, i: obj['Revenues'][i] / obj['Receivables'][i],
        'x62': lambda obj, i: obj['Current Liabilities'][i] * 365 / obj['Revenues'][i],
        'x63': lambda obj, i: obj['Revenues'][i] / obj['Current Liabilities'][i],
        'x64': lambda obj, i: obj['Revenues'][i] / obj['Total Noncurrent Assets'][i]
    }
    res = {ratio:[catchError(obj, i, funcs[ratio]) for i in range(len(obj['Share Price']))] for ratio in funcs}
    return res

In [1]:
x = get_last_24_features(data[0])

NameError: name 'get_last_24_features' is not defined

In [31]:
[a for a in x['x41'] if a != '']

[304.3097988949759,
 179.13805097951916,
 185.53047417206204,
 147.66199199029813,
 106.63625572874328,
 95.31247576204143,
 168.12392098606819,
 115.3864169708709,
 127.30508717895178,
 104.50805351669057,
 167.0958890406612,
 159.1538571882454,
 156.6221934369603,
 231.91687490584937,
 558.3494508661753,
 276.2185523231545,
 196.3181811376125,
 182.96592694422628,
 270.12743261709545,
 226.51730476105087,
 193.51390258964304,
 177.30162967277118,
 150.3814883518834,
 256.03861169931184,
 181.2212519418953,
 189.68550030858182,
 197.44076997921368]